In [ ]:
import tensorflow as tf

In [ ]:
t = tf.constant([[[1, 1, 1], [2, 2, 2]],
                 [[3, 3, 3], [4, 4, 4]],
                 [[5, 5, 5], [6, 6, 6]]])
t.shape

In [ ]:
a = tf.slice(t, [0,0,0], [1,1,3])

In [ ]:
a.shape, t.shape

In [ ]:
lists = []
for i in range(len(t)):
    a = tf.slice(t, [i,0,0], [1,1,3])
    lists.append(a)

In [ ]:
x = tf.concat(lists, 0)

In [ ]:
x

In [ ]:
a = tf.zeros([12, 384, 768])

In [ ]:
lists=[]
for i in range(len(a)):
    b = tf.slice(a, [i,0,0], [1, 384, 768])
    lists.append(b)

In [ ]:
tf.concat(lists, 0)

In [1]:
import tensorflow as tf
import math

In [9]:
tensor = tf.random.uniform([12,384,768],dtype=tf.float32)
logits = tf.random.uniform([12,384,1],dtype=tf.float32)

In [8]:
logits = tf.squeeze(logits, axis=-1)

In [10]:
tensor*logits

<tf.Tensor 'mul:0' shape=(12, 384, 768) dtype=float32>

In [ ]:
a = tf.zeros([12,12,114])
a.shape

In [ ]:
a = tf.expand_dims(a, 2)
a.shape

In [37]:
s = "2[abc]3[cd]ef"
tmp = s.split(']')

In [42]:
res = ''
for words in tmp:
    if '[' in words:
        digit, word = words.split('[')
        res += int(digit) * word
    else:
        res += words
res

'abcabccdcdcdef'

# Tensorflow

In [1168]:
def scoring(x1, x2):

    x1_rep = x1
    x2_rep = x2
    x2_rep = tf.transpose(x2_rep, perm=[0, 1, 3, 2])#? * 12 * 768 * 114
    scores = tf.einsum('abij,abjk->abik', x1_rep, x2_rep)
    return scores

In [1021]:
# input_tensor = tf.zeros([12,384,768])
# history_qa_ids = tf.zeros([12,12,114,768])
# history_mask = tf.zeros([12,12,114])

In [138]:
input_tensor_0 = tf.random.uniform([12,384,768],dtype=tf.float32)
history_qa_ids = tf.random.uniform([12,12,114,768],dtype=tf.float32)
history_mask = tf.random.uniform([12,12,114],dtype=tf.float32)
input_tensor = input_tensor_0

In [151]:
tf.compat.v1.disable_eager_execution()
x = tf.zeros([12,12,114])
x_max = tf.reduce_max(x)
with tf.compat.v1.Session() as sess:
    print(sess.run(x_max)==0)

True


In [1194]:
input_tensor = tf.expand_dims(input_tensor, 1)
input_tensor = tf.tile(input_tensor, [1, history_qa_ids.shape[1], 1, 1])#? * 12 * 384 * 768
input_tensor.shape

TensorShape([12, 12, 384, 768])

In [1195]:
scores = scoring(input_tensor, history_qa_ids)

In [144]:
history_mask = tf.zeros([12,12,114])
tmp = tf.cast(history_mask, tf.bool)

<tf.Tensor: shape=(), dtype=bool, numpy=False>

In [1196]:
# history_mask = tf.cast(history_mask,tf.bool)==False
history_mask = tf.expand_dims(history_mask, 2)
empty_mask = tf.tile(history_mask, [1, 1, scores.shape[2], 1])#矩阵扩张
score = tf.cast(scores, tf.float32)
empty_mask = tf.cast(empty_mask, tf.float32)
negmask = 1 - empty_mask
num = 3.4 * math.pow(10, 38)
score =  (score * empty_mask)
# + (-((negmask * num + num) - num))
score

<tf.Tensor: shape=(12, 12, 384, 114), dtype=float32, numpy=
array([[[[192.30557  , 127.09628  ,  16.344051 , ..., 144.0058   ,
           10.827468 ,   7.5531187],
         [189.6938   , 118.073296 ,  15.489044 , ..., 134.33255  ,
           10.60481  ,   7.188745 ],
         [184.3675   , 118.85626  ,  15.437724 , ..., 136.68878  ,
           10.476266 ,   7.273374 ],
         ...,
         [184.89989  , 118.7875   ,  15.358321 , ..., 136.0652   ,
           10.380463 ,   7.4600434],
         [195.59761  , 123.57488  ,  16.301638 , ..., 141.673    ,
           10.763184 ,   7.5090013],
         [192.25766  , 118.33291  ,  15.847146 , ..., 136.65858  ,
           10.591273 ,   7.3029113]],

        [[186.08284  , 121.251785 ,  42.65475  , ..., 200.69844  ,
           38.052063 ,  20.294739 ],
         [173.74956  , 112.26623  ,  40.44969  , ..., 187.29723  ,
           36.06563  ,  19.014906 ],
         [173.29851  , 116.26171  ,  40.397606 , ..., 192.17113  ,
           36.73207  ,  1

In [1197]:
score.shape

TensorShape([12, 12, 384, 114])

In [1198]:
score = tf.unstack(score, axis=1)
n = len(score)
score = tf.concat([i for i in score], 1)
score

<tf.Tensor: shape=(12, 4608, 114), dtype=float32, numpy=
array([[[192.30557  , 127.09628  ,  16.344051 , ..., 144.0058   ,
          10.827468 ,   7.5531187],
        [189.6938   , 118.073296 ,  15.489044 , ..., 134.33255  ,
          10.60481  ,   7.188745 ],
        [184.3675   , 118.85626  ,  15.437724 , ..., 136.68878  ,
          10.476266 ,   7.273374 ],
        ...,
        [148.69525  ,  61.39307  ,  76.54959  , ..., 112.06161  ,
          75.449425 , 187.42004  ],
        [151.10503  ,  64.36484  ,  80.42922  , ..., 114.31192  ,
          78.04297  , 191.01416  ],
        [149.9233   ,  61.8364   ,  77.93817  , ..., 110.579025 ,
          76.598564 , 184.83827  ]],

       [[165.42657  ,  46.63275  , 125.343155 , ...,  39.733128 ,
          58.297688 ,  57.13493  ],
        [169.03186  ,  46.364326 , 126.192406 , ...,  40.618126 ,
          58.03145  ,  58.11428  ],
        [163.80927  ,  46.24473  , 128.10054  , ...,  38.96541  ,
          58.29622  ,  57.76565  ],
        ..

In [1180]:
alpha_flat = tf.nn.softmax(score, axis=2)
alpha_flat = tf.split(alpha_flat, n, axis=1)
# alpha_flat

In [1181]:

alpha = tf.stack([i for i in alpha_flat], axis=1)
alpha.shape, alpha.shape


(TensorShape([12, 12, 384, 114]), TensorShape([12, 12, 384, 114]))

In [1182]:
x3 = history_qa_ids
attended = tf.matmul(alpha, x3)
attended


<tf.Tensor: shape=(12, 12, 384, 768), dtype=float32, numpy=
array([[[[0.8410448 , 0.13830543, 0.00114262, ..., 0.52358794,
          0.72532535, 0.971872  ],
         [0.8410448 , 0.13830543, 0.00114262, ..., 0.52358794,
          0.72532535, 0.971872  ],
         [0.8410448 , 0.13830543, 0.00114262, ..., 0.52358794,
          0.72532535, 0.971872  ],
         ...,
         [0.8410448 , 0.13830543, 0.00114262, ..., 0.52358794,
          0.72532535, 0.971872  ],
         [0.8410448 , 0.13830543, 0.00114262, ..., 0.52358794,
          0.72532535, 0.971872  ],
         [0.8410448 , 0.13830543, 0.00114262, ..., 0.52358794,
          0.72532535, 0.971872  ]],

        [[0.94210297, 0.41609344, 0.9002107 , ..., 0.05773615,
          0.46965045, 0.34663984],
         [0.94384927, 0.41795754, 0.90637714, ..., 0.05107086,
          0.47022146, 0.35164154],
         [0.8458577 , 0.4704825 , 0.8786709 , ..., 0.04699301,
          0.44729236, 0.3265707 ],
         ...,
         [0.9492508 , 0.4205

In [1064]:
tmp = tf.unstack(attended, axis=1)
n = len(tmp)


In [ ]:
mean = tmp[0]
for i in range(1, n):
    mean += tmp[i]
mean = tf.cast(mean, tf.float32)
attend = mean / n
attend.shape, attend

In [183]:
x = tf.random.uniform([1,384,768])
x= tf.squeeze(x, axis=0)
x.shape

TensorShape([384, 768])

In [1]:
import math
def scoring(x1, x2):

    x1_rep = x1#1 * 384 * 768
    x2_rep = x2#1 * 114 * 768

    # scores = x1_rep.bmm(x2_rep.transpose(1, 2))
    x2_rep = tf.transpose(x2_rep, perm=[0, 2, 1])
    scores = tf.matmul(x1_rep, x2_rep)#1 * 384 * 114
    return scores

In [3]:
input_tensor = tf.random.uniform([12,384,768])
h1 = tf.random.uniform([12,64,768])
h2 = tf.zeros([12,208,768])
history = tf.concat([h1,h2], axis=1)

h1 = tf.ones([12,64])
h2 = tf.zeros([12,208])
mask = tf.concat([h1,h2], axis=1)

In [4]:
with tf.Session() as sess:
    print(sess.run(input_tensor))

[[[0.97554016 0.2138685  0.4384668  ... 0.7832124  0.8633803  0.20647144]
  [0.6306399  0.14853203 0.49749684 ... 0.7296274  0.76150525 0.06837916]
  [0.45558774 0.7244091  0.8362372  ... 0.3110727  0.6083883  0.08332443]
  ...
  [0.45657325 0.02335835 0.5152613  ... 0.3923632  0.5956837  0.04495895]
  [0.03296769 0.7556968  0.94366395 ... 0.82735443 0.8621496  0.65742266]
  [0.9088532  0.49004745 0.62592053 ... 0.5901555  0.73852825 0.33289838]]

 [[0.20606971 0.45240033 0.05021191 ... 0.20194292 0.45343554 0.9482714 ]
  [0.9467952  0.63757205 0.3852414  ... 0.14810884 0.00824761 0.74758506]
  [0.9879557  0.3669536  0.19919884 ... 0.26906836 0.12537587 0.1531595 ]
  ...
  [0.1709044  0.4179517  0.00515926 ... 0.13712788 0.8612112  0.58745146]
  [0.23606515 0.30902517 0.9543301  ... 0.3784498  0.17841649 0.0513016 ]
  [0.12174988 0.88315105 0.71752346 ... 0.70164454 0.34793198 0.845577  ]]

 [[0.7108731  0.07919455 0.86368597 ... 0.89878774 0.27103567 0.473184  ]
  [0.6090425  0.046349

In [112]:
mask = ~tf.cast(mask, tf.bool)
mask

<tf.Tensor 'LogicalNot_3:0' shape=(12, 272) dtype=bool>

In [5]:
def mask_fill_inf(matrix, mask):
    negmask = 1 - mask
    num = 3.4 * math.pow(10, 38)
    return (matrix * mask) + (-((negmask * num + num) - num))

In [6]:

scores = scoring(input_tensor, history)
mask = tf.tile(tf.expand_dims(mask, axis=1), [1, 384, 1])
scores = mask_fill_inf(scores, mask)

# empty_mask = tf.tile(mask, [1, scores.shape[1], 1])#矩阵扩张 12，12，384，114
# score = tf.cast(scores, tf.float32)
# empty_mask = tf.cast(empty_mask, tf.float32)
scores = tf.reshape(scores, [-1, 272])
alpha_flat = tf.nn.softmax(scores, axis=1)
alpha = tf.reshape(alpha_flat, [-1, 384, 272])
attend = tf.matmul(alpha, history)
attend

<tf.Tensor 'MatMul_1:0' shape=(12, 384, 768) dtype=float32>

In [7]:
with tf.Session() as sess:
    print(sess.run(attend))

[[[0.5112139  0.96938694 0.828981   ... 0.6151375  0.18611401 0.15173799]
  [0.68687284 0.778277   0.43345213 ... 0.5198698  0.38006687 0.5942273 ]
  [0.45146805 0.714343   0.86227185 ... 0.60721296 0.31994036 0.2914255 ]
  ...
  [0.5458357  0.6607005  0.6211204  ... 0.55533254 0.40247008 0.3825738 ]
  [0.5060566  0.98119444 0.84461886 ... 0.6170002  0.17991577 0.14714915]
  [0.7933887  0.70581114 0.57987714 ... 0.23000601 0.49147522 0.33478218]]

 [[0.65834147 0.632716   0.24272782 ... 0.85494405 0.7245021  0.08654165]
  [0.6065849  0.6122038  0.24836095 ... 0.8260442  0.70458883 0.1148698 ]
  [0.46667635 0.6144969  0.39790982 ... 0.7130008  0.48890018 0.45273292]
  ...
  [0.08696221 0.42302507 0.47148317 ... 0.6060282  0.36520064 0.2826589 ]
  [0.5365677  0.95083153 0.6863917  ... 0.86177826 0.17812468 0.3139429 ]
  [0.63767415 0.63929355 0.2406481  ... 0.8408814  0.72354865 0.09266835]]

 [[0.40365347 0.09671991 0.53499097 ... 0.6887844  0.5393072  0.6245417 ]
  [0.27352807 0.209351

In [60]:
# tf.reduce_mean(score, axis=1)
int(mask[0][0][0])

0

In [203]:
def scoring(x1, x2):
    '''
        Input:
        x1: batch x word_num1 x dim
        x2: batch x word_num2 x dim
        Output:
        scores: batch x word_num1 x word_num2
        '''
    # x1 = dropout(x1, p = dropout_p, training = self.training)
    # x2 = dropout(x2, p = dropout_p, training = self.training)
    x1_rep = x1#4 * 384 * 768
    x2_rep = x2#4 * 114 * 768

    # scores = x1_rep.bmm(x2_rep.transpose(1, 2))
    x2_rep = tf.transpose(x2_rep, perm=[0, 2, 1])
    scores = tf.einsum('aik,akj->aij', x1_rep, x2_rep)#4 * 384 * 114
    # x2_rep = tf.transpose(x2_rep, perm=[0, 1, 3, 2])#? * 12 * 768 * 114
    # scores = tf.einsum('abij,abjk->abik', x1_rep, x2_rep)#? * 12 * 384 * 114
    
    return scores

In [231]:
input_tensor = tf.random.uniform([4,384,768])
history_output = tf.random.uniform([4,12,114,768])
history_mask = tf.random.uniform([4,12,114])

num1 = history_output.shape[0]
num2 = history_output.shape[1]

single_inputs = tf.split(input_tensor, num1, axis=0)
single_historys = tf.split(history_output, num1, axis=0)
single_masks = tf.split(history_mask, num1, axis=0)
# print(single_inputs[0].shape)
new_input = []
for single_input, single_history, single_mask in zip(single_inputs, single_historys, single_masks):
    each_historys = tf.split(single_history, num2, axis=1)
 
    each_masks = tf.split(single_mask, num2, axis=1)
    new_his = []
    new_mask = []
    for each_history, each_mask in zip(each_historys, each_masks):
        each_history = tf.squeeze(each_history, axis=1)
        each_mask = tf.squeeze(each_mask, axis=1)
        new_his.append(each_history)#把不是0的history找出来
        new_mask.append(each_mask)
#     print(new_his[0].shape)
    history = tf.concat([i for i in new_his], axis=0)
    mask = tf.concat([i for i in new_mask], axis=0)
    single_input = tf.tile(single_input, [history.shape[0], 1, 1])
    
#     print("single_input: ", single_input.shape)
#     print("history: ", history.shape)
    scores = scoring(single_input, history)
    new_mask = tf.expand_dims(mask, 1)#4，1，114
    empty_mask = tf.tile(new_mask, [1, scores.shape[1], 1])#矩阵扩张 12，12，384，114
    score = tf.cast(scores, tf.float32)
    empty_mask = tf.cast(empty_mask, tf.float32)#4, 384, 114

    score =  (score * empty_mask)#4, 384, 114
    score = tf.unstack(score, axis=0)
    n = len(score)
    score = tf.concat([i for i in score], 0)
    score.shape

    alpha_flat = tf.nn.softmax(score, axis=1)
    alpha_flat = tf.split(alpha_flat, n, axis=0)
    alpha = tf.stack([i for i in alpha_flat], axis=0)

    attended = tf.matmul(alpha, history)
    tmp = tf.unstack(attended, axis=0)
    n = len(tmp)
    mean = tmp[0]
    for i in range(1, n):
        mean += tmp[i]
    mean = tf.cast(mean, tf.float32)

    # dim = list(map(float, history_qa_ids.shape))
    attend = mean / n
    new_input.append(attend)
attend_tensor = tf.stack([i for i in new_input], axis=0)
print(attend_tensor.shape)
# sum(attend_tensor[0][0][0]) 
# attend_tensor
with tf.compat.v1.Session() as sess:
    print(sess.run((tf.reduce_sum(attend_tensor))))
    print(sess.run((tf.reduce_sum(input_tensor))))


(4, 384, 768)
599748.0
590227.9


In [198]:
history = tf.concat([i for i in new_his], axis=0)
history.shape

TensorShape([12, 114, 768])

# Pytorch


In [65]:
def forward(x1, x2):

        x1_rep = x1
        x2_rep = x2

        scores = x1_rep.matmul(x2_rep.transpose(2,1))
        return scores

In [15]:
import torch
import torch.nn.functional as F
# a=torch.Tensor(2,3).uniform_(-1,1)

In [136]:
input_tensor = torch.rand([12,384,768])
history_qa_ids = torch.ones([12,114,768])
history_mask = torch.zeros([12,114])
x3 = history_qa_ids
history_qa_ids.shape

torch.Size([12, 114, 768])

In [135]:
history_mask.shape

torch.Size([12, 114, 768])

In [133]:
scores = forward(input_tensor, history_qa_ids)
scores.shape

torch.Size([12, 384, 114])

In [134]:
empty_mask = history_mask.eq(0).unsqueeze(1).expand_as(scores)
# scores.data.masked_fill_(empty_mask.data, -float('inf'))
alpha_flat = F.softmax(scores.view(12, -1, history_qa_ids.size(2)), dim = 2)
alpha = alpha_flat.view(-1,12, 384, 114)

RuntimeError: expand(CPUBoolType{[12, 1, 114, 768]}, size=[12, 384, 114]): the number of sizes provided (3) must be greater or equal to the number of dimensions in the tensor (4)

In [118]:
alpha.shape


torch.Size([1, 12, 384, 114])

In [119]:
scores.view(12, -1, history_qa_ids.size(2)).shape

torch.Size([12, 57, 768])

In [120]:
alpha_flat.shape, alpha.shape, x3.shape

(torch.Size([12, 57, 768]),
 torch.Size([1, 12, 384, 114]),
 torch.Size([12, 114, 768]))

In [121]:
attended = alpha.matmul(x3)
attended.shape

torch.Size([1, 12, 384, 768])

In [122]:
attended

tensor([[[[1.1584e-02, 1.1584e-02, 1.1584e-02,  ..., 1.1584e-02,
           1.1584e-02, 1.1584e-02],
          [1.4302e-03, 1.4302e-03, 1.4302e-03,  ..., 1.4302e-03,
           1.4302e-03, 1.4302e-03],
          [8.1596e-07, 8.1596e-07, 8.1596e-07,  ..., 8.1596e-07,
           8.1596e-07, 8.1596e-07],
          ...,
          [8.2477e-04, 8.2477e-04, 8.2477e-04,  ..., 8.2477e-04,
           8.2477e-04, 8.2477e-04],
          [3.5987e-06, 3.5987e-06, 3.5987e-06,  ..., 3.5987e-06,
           3.5987e-06, 3.5987e-06],
          [8.3319e-02, 8.3319e-02, 8.3319e-02,  ..., 8.3319e-02,
           8.3319e-02, 8.3319e-02]],

         [[2.9893e-06, 2.9893e-06, 2.9893e-06,  ..., 2.9893e-06,
           2.9893e-06, 2.9893e-06],
          [6.9360e-07, 6.9360e-07, 6.9360e-07,  ..., 6.9360e-07,
           6.9360e-07, 6.9360e-07],
          [1.8963e-09, 1.8963e-09, 1.8963e-09,  ..., 1.8963e-09,
           1.8963e-09, 1.8963e-09],
          ...,
          [1.1989e-05, 1.1989e-05, 1.1989e-05,  ..., 1.1989

In [ ]:
sum(x1[0][0])

In [123]:
sum(attended[0][0][0])

tensor(8.8967)

In [ ]:
x1 = torch.rand([12,384,768])
x2 = torch.rand([12,12,114,768])
x2_mask = torch.rand([12,12,114])

In [ ]:
x3 = x2
x = torch.zeros([12,12,384,768])
x1 = x1.unsqueeze(1).expand_as(x)

In [ ]:
x1_rep = x1
x2_rep = x2
scores = x1_rep.matmul(x2_rep.transpose(2, 3))

In [ ]:
scores.shape

In [ ]:
empty_mask = x2_mask.eq(0).unsqueeze(2).expand_as(scores)
scores.data.masked_fill_(empty_mask.data, -float('inf'))
scores.shape

In [ ]:
scores

In [ ]:
alpha_flat = F.softmax(scores.view(-1, x2.size(1)), dim = 1)
alpha = alpha_flat.view(-1, x1.size(1), x2.size(1))
alpha_flat.shape, alpha.shape

In [ ]:
x3.shape

In [ ]:
alpha

In [ ]:
attended = alpha.bmm(x3)
attended

In [ ]:
attended = alpha.matmul(x3)
attended

In [ ]:
attended

In [ ]:
sum(x1[0][0])

In [ ]:
sum(attended[0][0])

In [1052]:
x = torch.rand([384,768])
x = x.expand([12,-1,-1])
x.shape

torch.Size([12, 384, 768])

In [25]:
def new(x1, x2):

        x1_rep = x1
        x2_rep = x2

        scores = x1_rep.matmul(x2_rep.transpose(2,1))
        return scores

In [34]:
a = torch.rand([12,384,768])
b = torch.rand([12,64,768])
c = torch.zeros([12,208,768])
# b = a
d = torch.cat([b,c], axis=1)
# c = torch.ones([12,50]).bool()
t1 = torch.ones([12,64])
t2 = torch.zeros([12,208])
mask = torch.cat([t1,t2], axis=1)

In [38]:
scores = new(a, d)
empty_mask = mask.eq(0).unsqueeze(1).expand_as(scores)
scores.data.masked_fill_(empty_mask.data, -float('inf'))
alpha_flat = F.softmax(scores.view(-1, d.size(1)), dim = 1)
alpha = alpha_flat.view(-1, 384, 272)
attended = alpha.matmul(d)
attended.size()

torch.Size([12, 384, 768])

In [286]:
scores.shape

torch.Size([12, 100, 50])

In [287]:
x3 = torch.rand([12,114,768])
x3

tensor([[[1.3005e-01, 8.1368e-01, 6.3015e-01,  ..., 6.0221e-01,
          9.4395e-01, 8.1164e-01],
         [5.8506e-01, 7.9978e-01, 7.4815e-01,  ..., 5.0843e-01,
          8.7747e-01, 2.7704e-01],
         [1.3238e-01, 5.1905e-01, 7.1893e-02,  ..., 5.0848e-01,
          1.1791e-01, 5.0154e-02],
         ...,
         [6.2830e-01, 4.8316e-02, 4.4810e-01,  ..., 6.4548e-02,
          2.5952e-01, 6.6925e-02],
         [9.2278e-01, 5.7031e-01, 5.0595e-01,  ..., 5.3350e-01,
          9.3800e-01, 2.8004e-01],
         [3.6305e-01, 5.8494e-01, 5.4665e-01,  ..., 1.6257e-01,
          5.6919e-01, 7.9708e-01]],

        [[2.7301e-02, 3.0565e-01, 3.8107e-01,  ..., 7.2160e-01,
          7.4898e-01, 7.3716e-01],
         [4.1308e-01, 2.7185e-01, 6.2480e-01,  ..., 5.1992e-01,
          7.1710e-01, 2.6360e-01],
         [8.9103e-01, 1.0473e-01, 3.2865e-01,  ..., 1.0153e-01,
          4.6308e-02, 8.6007e-01],
         ...,
         [4.2556e-01, 8.2972e-01, 6.9587e-01,  ..., 9.2039e-02,
          1.089

In [288]:
alpha = torch.zeros([1,384,384])

In [245]:
attend = torch.matmul(alpha, x3)

In [246]:
attend.shape

torch.Size([1, 384, 768])

In [247]:
attend

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])

In [356]:
a = tf.constant([12,4,5,6,7,9,6,3,2,4,6,7])
a

<tf.Tensor 'Const_808:0' shape=(12,) dtype=int32>

In [349]:
b = tf.split(a, 12, axis=0)

In [354]:
b[0]

<tf.Tensor 'split_782:0' shape=(1,) dtype=int32>

In [359]:
x = 4
tmp = tf.zeros([4,114,768])
lists = []
lists.append(tmp)

In [360]:
v2=tf.Variable(tf.constant(2),name='v2')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
@tf.function
def func(fn, tensor):
  return tf.map_fn(fn, tensor)

In [480]:
import numpy as np
# elems = np.array([1, 2, 3, 4, 5, 6])
elems = tf.constant([1,2,3,4,5], dtype=tf.int32)
tmp = tf.zeros([12, 12,114], dtype=tf.int32)
# tf.slice(tmp, [0,0,0],[1, 114,768])
du = tf.map_fn(lambda x: x*x, elems, dtype=tf.int32)
xxx = tf.constant([1,2,3,4,5])
# squares = tf.map_fn(lambda x, y: tf.slice(tmp, [y-1,0,0],[1, x, 114]), elems,xxx)
squares = func()
# squares == [1, 4, 9, 16, 25, 36]
squares.shape

OperatorNotAllowedInGraphError: using a `tf.Tensor` as a Python `bool` is not allowed in Graph execution. Use Eager execution or decorate this function with @tf.function.

In [454]:
new = tf.unstack(squares, 12, axis=0)
type

list

In [452]:
a = tf.ones([12,384,114])
a = tf.reduce_mean(a, axis=0)
tf.

TensorShape([384, 114])

In [451]:
with tf.compat.v1.Session() as sess:
    print(sess.run(str(b)))

ValueError: Fetch argument 'Tensor("Shape:0", shape=(2,), dtype=int32)' cannot be interpreted as a Tensor. (The name 'Tensor("Shape:0", shape=(2,), dtype=int32)' looks a like a Tensor name, but is not a valid one. Tensor names must be of the form "<op_name>:<output_index>".)

In [419]:
new = tf.unstack(squares, 12, axis=0)
# new = tf.split(squares, 12, axis=0)

new = new[0]
new

<tf.Tensor 'unstack_401:0' shape=(1, None, 114, 768) dtype=float32>

In [420]:
tf.squeeze(new, axis=0)

<tf.Tensor 'Squeeze_5716:0' shape=(None, 114, 768) dtype=float32>

In [465]:
lists = [1,2,3,4,5]

In [440]:
lists.pop(-1)
lists

[1, 2, 3, 4]

In [468]:
a = tf.ones([12,384,114])
b= tf.reshape(a, [-1,114])


AttributeError: 'Tensor' object has no attribute 'read'

In [471]:
tmp = tf.zeros([12,12,114], dtype=tf.int32)
TensorArr = tf.TensorArray(tf.int32, 1, dynamic_size=True, infer_shape=False)
x = TensorArr.unstack(tmp)
type(x.read(0))

(tensorflow.python.framework.ops.Tensor,
 tensorflow.python.framework.ops.Tensor)

In [492]:
@tf.function
def fun(tmp, tmp2, lis):
    new = []
    for t in tmp2:
        new.append(tf.slice(tmp, [0,0], [t,tmp.shape[1]]))
    return new

In [493]:
tmp = tf.zeros([12,114], dtype=tf.int32)
tmp2 = tf.constant([2,1,0,4,1])
lis = []
new = fun(tmp, tmp2, lis)

InaccessibleTensorError: The tensor 'Tensor("Slice:0", shape=(None, 114), dtype=int32)' cannot be accessed here: it is defined in another function or code block. Use return values, explicit Python locals or TensorFlow collections to access it. Defined in: FuncGraph(name=while_body_913617, id=6478056272); accessed from: FuncGraph(name=fun, id=6345570192).


In [517]:
history_output = tf.zeros([12,12,114,768])

fn = lambda x, y: tf.slice(history_output, [x,0,0,0],[1, y, 114, 768])

elems1 = tf.constant([1,2,3,4,3,5,6,7,3,2,1,4])
elems2 = tf.constant([0,1,2,3,4,5,6,7,8,9,10,11])
history_list = tf.unstack(tf.map_fn(fn, elems1, elems2), 12, axis=0)



OperatorNotAllowedInGraphError: using a `tf.Tensor` as a Python `bool` is not allowed in Graph execution. Use Eager execution or decorate this function with @tf.function.

In [523]:
nums = [list(range(i * 10, (i + 1) * 10)) + [0] for i in range(1, 5)]
matrix = tf.constant(nums)
X = tf.constant(36, dtype=matrix.dtype)

In [525]:
X.shape

TensorShape([])

In [528]:

a = tf.zeros([1, 4, 114], dtype=tf.float32)
b = tf.ones([1, 8, 114],dtype=tf.float32)
c = tf.concat([b, a], axis=1)

In [545]:
tmp = tf.unstack(c, axis=1)
fn = lambda x : x
compare = tf.zeros([1, 1, 114], dtype=tf.float32)
lists = []
for t in tmp:
    x = tf.reduce_mean(t)
    if x>0:
        lists.append(t)
#     result = tf.cond(t > y, tf.reduce_mean(t), tf.reduce_mean(compare))
#     print(result)
        

# with tf.compat.v1.Session() as sess:
#     for t in tmp:
#         x = tf.reduce_mean(t)
#         print(sess.run(x))

OperatorNotAllowedInGraphError: using a `tf.Tensor` as a Python `bool` is not allowed in Graph execution. Use Eager execution or decorate this function with @tf.function.

In [551]:
zeroes = tf.zeros([600, 1], dtype=tf.float32)
ones = tf.ones([600, 1], dtype=tf.float32)
b = tf.random.uniform([600,10], minval=0, maxval=1, dtype=tf.float32)
threshold = tf.constant(0., dtype=tf.float32)

check = tf.reduce_max(tf.cast(b > threshold, dtype=tf.float32), axis=1)
last_col = tf.where(check>0, zeroes, ones)
# new_b = tf.where(check>0, b, tf.zeros([600, 10], dtype=tf.float32))
# new_matrix = tf.concat([new_b, last_col], axis=1)

In [552]:
check.shape, last_col.shape

(TensorShape([600]), TensorShape([600, 600]))

In [553]:
b.shape

TensorShape([600, 10])

In [595]:
def func(i):
    return i

In [14]:
a = tf.zeros([1, 4, 114], dtype=tf.int32)
b = tf.ones([1, 8, 114], dtype=tf.int32)
c = tf.concat([b, a], axis=1)
d = tf.constant([1,1,1,1,1,1,1,1,0,0,0,0])

with tf.compat.v1.Session() as sess:
#     arr1 = tf.constant([[1,2],
#                         [3,4]])
#     arr2 = tf.constant([[False, True],
#                         [True, False]])
    print(sess.run(tf.boolean_mask(c, d)))

RuntimeError: Attempting to capture an EagerTensor without building a function.

In [13]:
a = tf.zeros([1, 4, 114], dtype=tf.int32)
b = tf.ones([1, 8, 114], dtype=tf.int32)
c = tf.concat([b, a], axis=1)
d = tf.constant([1,1,1,1,1,1,1,1,0,0,0,0])
# e = tf.zeros([1, 114], dtype=tf.int32)
# tmp = tf.unstack(c, axis=1)
# threshold = tf.constant(0, dtype=tf.int32)
# l = []
# s = []

# for i in tmp:
#     new = tf.reduce_max(i)
#     greater = tf.greater(new, threshold)
#     x = tf.cond(tf.equal(greater, tf.constant(True)), lambda : tf.concat([e,i],0), lambda : func(e))
# #     print(x.shape(0))
#     print(x)
d = tf.expand_dims(d,axis=0)
# tf.split(c,d,0)
c


<tf.Tensor: shape=(1, 12, 114), dtype=int32, numpy=
array([[[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]], dtype=int32)>

In [ ]:

with tf.compat.v1.Session() as sess:
    print(sess.run(l))

<tf.Tensor 'cond_15/Identity:0' shape=() dtype=int32>

In [569]:
a = tf.compat.v1.placeholder(tf.bool)  #placeholder for a single boolean value
b = tf.cond(tf.equal(a, tf.constant(True)), lambda: tf.constant(10), lambda: tf.constant(0))
sess = tf.compat.v1.InteractiveSession()
res = sess.run(b, feed_dict = {a: True})
sess.close()
print(res)

10


In [31]:
a = tf.zeros([1,12,114])
len(tf.unstack(a, 12, axis=1)[:turn])

5

In [36]:
history = tf.zeros([1,114,768])
mask = tf.zeros([1,114])
scores = tf.zeros([1,384,114])#1 * 384 * 114
num = tf.shape(scores)[1]
# empty_mask = history_mask.eq(0).unsqueeze(1).expand_as(scores)
history_mask = tf.expand_dims(mask, 1)#1，1，114
empty_mask = tf.tile(history_mask, [1, num, 1])#矩阵扩张 12，12，384，114
score = tf.cast(scores, tf.float32)
empty_mask = tf.cast(empty_mask, tf.float32)

score =  (score * empty_mask)

# softmax
# alpha = alpha_flat.view(-1, input_tensor.size(1), history_qa_ids.size(1))
alpha_flat = tf.nn.softmax(score, axis=2)
# alpha_flat = tf.split(alpha_flat, n, axis=0)
# alpha = tf.stack([i for i in alpha_flat], axis=0)

# alpha: batch x word_num1 x word_num2
attended = tf.matmul(alpha_flat, history)

In [42]:
attended.shape

TensorShape([1, 384, 768])

In [44]:
mean = tf.zeros(tf.shape(attended), dtype=tf.int32)

In [47]:
tf.squeeze(mean, axis=0)

<tf.Tensor: shape=(384, 768), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>

In [48]:
tensor = [101, 2339, 2106, 2002, 2689, 2010, 2490, 2000, 2068, 1029, 102, 17251, 13198, 2031, 2036, 4427, 2195, 4684, 5097, 2005, 16380, 1998, 11924, 1012, 4401, 1010, 2107, 2004, 16480, 14115, 26760, 24079, 1010, 2031, 7034, 2008, 11948, 5498, 9126, 2705, 2038, 1996, 4022, 2000, 2022, 3144, 1999, 2796, 4331, 2349, 2000, 2010, 6217, 1998, 5470, 2918, 2894, 1012, 1999, 2786, 1010, 11948, 5498, 9126, 2705, 2211, 4637, 1996, 2796, 2120, 3519, 2044, 3116, 3539, 2704, 1052, 1012, 1058, 1012, 27544, 5332, 2213, 3270, 10546, 1012, 2019, 5448, 8554, 4146, 2011, 1996, 2932, 13970, 12274, 17130, 10173, 2008, 3519, 1010, 2007, 11948, 5498, 9126, 2705, 1005, 1055, 2490, 1010, 2453, 2663, 2039, 2000, 7558, 4272, 1999, 1996, 6008, 10703, 3320, 1012, 1999, 2727, 1010, 2043, 1996, 3519, 2283, 2787, 2000, 25705, 2007, 2035, 2634, 4698, 2852, 18891, 2850, 14163, 10087, 6494, 10556, 27922, 16098, 2213, 1006, 9932, 4215, 2213, 2243, 1007, 2005, 1996, 3320, 2602, 1999, 6008, 10703, 1010, 11948, 5498, 9126, 2705, 2904, 8884, 7368, 1998, 3569, 1996, 2852, 18891, 2850, 14163, 10087, 6494, 10556, 27922, 16098, 2213, 1006, 1040, 2213, 2243, 1007, 1011, 6008, 5003, 7088, 2721, 3519, 1006, 1056, 12458, 1007, 4707, 1012, 1996, 1056, 12458, 2109, 1037, 10165, 2004, 2037, 2602, 6454, 1998, 2109, 2019, 3746, 1997, 11948, 5498, 9126, 2705, 5559, 1037, 10165, 2013, 1996, 2143, 4698, 9067, 4886, 1999, 2037, 14921, 1012, 11948, 5498, 9126, 2705, 2056, 1024, 1000, 2130, 2643, 3685, 3828, 6008, 10703, 2065, 9932, 4215, 2213, 2243, 5651, 2000, 2373, 1012, 1000, 11948, 5498, 9126, 2705, 2878, 27693, 2135, 3569, 1996, 1040, 2213, 2243, 1998, 1056, 12458, 4707, 1998, 2356, 1996, 2111, 1997, 6008, 10703, 1998, 2010, 4599, 2000, 3789, 2005, 2008, 4707, 1012, 2023, 4707, 2018, 1037, 3143, 3377, 1999, 2727, 1012, 11948, 5498, 9126, 2705, 2036, 3569, 1996, 1040, 2213, 2243, 1011, 1056, 12458, 4707, 1999, 1996, 5768, 2602, 1010, 2218, 1996, 2168, 2095, 1012, 2101, 1999, 2432, 1010, 11948, 5498, 9126, 2705, 2056, 2002, 2052, 7714, 3789, 2005, 1996, 24243, 20308, 2283, 1006, 24954, 1007, 2021, 2052, 2025, 7949, 2010, 2490, 2000, 2151, 2392, 2076, 1996, 9046, 2796, 2236, 2602, 1012, 1996, 2283, 1010, 2174, 1010, 3478, 2000, 2663, 2151, 4272, 1999, 6008, 10703, 1999, 1996, 13660, 11200, 1012, 4599, 1997, 11948, 5498, 9126, 2705, 1999, 6008, 10703, 2031, 10843, 15520, 2006, 102]


In [56]:
import pandas as pd
input_id = pd.DataFrame(tensor)

In [127]:
from torch.autograd import Variable
import torch
from torch.nn import functional 
embedding_table = Variable(torch.randn([32000, 128],dtype=float))

In [128]:
embedding_table

tensor([[-0.5388, -0.0370, -0.2933,  ...,  0.6444,  1.4417,  1.3879],
        [-0.2459,  1.2643,  0.1578,  ...,  0.2839, -0.7094, -1.2146],
        [-1.2718,  1.3095,  1.0598,  ..., -0.3237,  0.1601, -0.1534],
        ...,
        [ 0.2182,  0.6415,  0.0186,  ...,  0.8556,  0.5961, -0.0383],
        [-0.6343,  2.8066, -2.1052,  ...,  0.1330, -0.8524,  1.2732],
        [ 0.1418, -0.6734,  0.0218,  ..., -1.3258, -1.3943, -0.9412]],
       dtype=torch.float64)

In [133]:
def create_initializer(initializer_range=0.02):
  """Creates a `truncated_normal_initializer` with the given range."""
  return tf.compat.v1.truncated_normal_initializer(stddev=initializer_range)


In [145]:
embedding_table = tf.compat.v1.get_variable(
      name='word_embedding_name',
      shape=[32000, 128],
      initializer=create_initializer(0.02),
      dtype=tf.float64)


In [147]:
flat_input_ids = tf.reshape(input_ids, [-1])
one_hot_input_ids = tf.one_hot(flat_input_ids, depth=32000)
# output = tf.matmul(one_hot_input_ids, embedding_table)



NotFoundError: Could not find valid device for node.
Node:{{node OneHot}}
All kernels registered for op OneHot :
  device='XLA_CPU_JIT'; TI in [DT_INT32, DT_UINT8, DT_INT64]; T in [DT_FLOAT, DT_DOUBLE, DT_INT32, DT_UINT8, DT_INT16, ..., DT_UINT16, DT_COMPLEX128, DT_HALF, DT_UINT32, DT_UINT64]
  device='XLA_CPU'; TI in [DT_INT32, DT_UINT8, DT_INT64]; T in [DT_FLOAT, DT_DOUBLE, DT_INT32, DT_UINT8, DT_INT16, ..., DT_UINT16, DT_COMPLEX128, DT_HALF, DT_UINT32, DT_UINT64]
  device='CPU'; TI in [DT_UINT8]; T in [DT_INT64]
  device='CPU'; TI in [DT_INT32]; T in [DT_INT64]
  device='CPU'; TI in [DT_INT64]; T in [DT_INT64]
  device='CPU'; TI in [DT_UINT8]; T in [DT_INT32]
  device='CPU'; TI in [DT_INT32]; T in [DT_INT32]
  device='CPU'; TI in [DT_INT64]; T in [DT_INT32]
  device='CPU'; TI in [DT_UINT8]; T in [DT_UINT16]
  device='CPU'; TI in [DT_INT32]; T in [DT_UINT16]
  device='CPU'; TI in [DT_INT64]; T in [DT_UINT16]
  device='CPU'; TI in [DT_UINT8]; T in [DT_INT16]
  device='CPU'; TI in [DT_

In [129]:
# input_ids = torch.ones([384])
# input_ids = torch.tensor([])
flat_input_ids = torch.reshape(input_ids, [-1])
one_hot_input_ids = functional.one_hot(flat_input_ids, num_classes=32000)
output = torch.matmul(one_hot_input_ids, embedding_table)

RuntimeError: one_hot is only applicable to index tensor.

In [7]:
lis = [101, 2339, 2106, 2002, 2689, 2010, 2490, 2000, 2068, 1029, 102, 17251, 13198, 2031, 2036, 4427, 2195, 4684, 5097, 2005, 16380, 1998, 11924, 1012, 4401, 1010, 2107, 2004, 16480, 14115, 26760, 24079, 1010, 2031, 7034, 2008, 11948, 5498, 9126, 2705, 2038, 1996, 4022, 2000, 2022, 3144, 1999, 2796, 4331, 2349, 2000, 2010, 6217, 1998, 5470, 2918, 2894, 1012, 1999, 2786, 1010, 11948, 5498, 9126, 2705, 2211, 4637, 1996, 2796, 2120, 3519, 2044, 3116, 3539, 2704, 1052, 1012, 1058, 1012, 27544, 5332, 2213, 3270, 10546, 1012, 2019, 5448, 8554, 4146, 2011, 1996, 2932, 13970, 12274, 17130, 10173, 2008, 3519, 1010, 2007, 11948, 5498, 9126, 2705, 1005, 1055, 2490, 1010, 2453, 2663, 2039, 2000, 7558, 4272, 1999, 1996, 6008, 10703, 3320, 1012, 1999, 2727, 1010, 2043, 1996, 3519, 2283, 2787, 2000, 25705, 2007, 2035, 2634, 4698, 2852, 18891, 2850, 14163, 10087, 6494, 10556, 27922, 16098, 2213, 1006, 9932, 4215, 2213, 2243, 1007, 2005, 1996, 3320, 2602, 1999, 6008, 10703, 1010, 11948, 5498, 9126, 2705, 2904, 8884, 7368, 1998, 3569, 1996, 2852, 18891, 2850, 14163, 10087, 6494, 10556, 27922, 16098, 2213, 1006, 1040, 2213, 2243, 1007, 1011, 6008, 5003, 7088, 2721, 3519, 1006, 1056, 12458, 1007, 4707, 1012, 1996, 1056, 12458, 2109, 1037, 10165, 2004, 2037, 2602, 6454, 1998, 2109, 2019, 3746, 1997, 11948, 5498, 9126, 2705, 5559, 1037, 10165, 2013, 1996, 2143, 4698, 9067, 4886, 1999, 2037, 14921, 1012, 11948, 5498, 9126, 2705, 2056, 1024, 1000, 2130, 2643, 3685, 3828, 6008, 10703, 2065, 9932, 4215, 2213, 2243, 5651, 2000, 2373, 1012, 1000, 11948, 5498, 9126, 2705, 2878, 27693, 2135, 3569, 1996, 1040, 2213, 2243, 1998, 1056, 12458, 4707, 1998, 2356, 1996, 2111, 1997, 6008, 10703, 1998, 2010, 4599, 2000, 3789, 2005, 2008, 4707, 1012, 2023, 4707, 2018, 1037, 3143, 3377, 1999, 2727, 1012, 11948, 5498, 9126, 2705, 2036, 3569, 1996, 1040, 2213, 2243, 1011, 1056, 12458, 4707, 1999, 1996, 5768, 2602, 1010, 2218, 1996, 2168, 2095, 1012, 2101, 1999, 2432, 1010, 11948, 5498, 9126, 2705, 2056, 2002, 2052, 7714, 3789, 2005, 1996, 24243, 20308, 2283, 1006, 24954, 1007, 2021, 2052, 2025, 7949, 2010, 2490, 2000, 2151, 2392, 2076, 1996, 9046, 2796, 2236, 2602, 1012, 1996, 2283, 1010, 2174, 1010, 3478, 2000, 2663, 2151, 4272, 1999, 6008, 10703, 1999, 1996, 13660, 11200, 1012, 4599, 1997, 11948, 5498, 9126, 2705, 1999, 6008, 10703, 2031, 10843, 15520, 2006, 102]
print(len(lis))
input_id= tf.constant(lis,dtype=tf.float64)





384


In [3]:
input_id=[101.6, 2339, 2106, 2002, 2689, 2010, 2490, 2000, 2068, 1029, 102, 17251, 13198, 2031, 2036, 4427, 2195, 4684, 5097, 2005, 16380, 1998, 11924, 1012, 4401, 1010, 2107, 2004, 16480, 14115, 26760, 24079, 1010, 2031, 7034, 2008, 11948, 5498, 9126, 2705, 2038, 1996, 4022, 2000, 2022, 3144, 1999, 2796, 4331, 2349, 2000, 2010, 6217, 1998, 5470, 2918, 2894, 1012, 1999, 2786, 1010, 11948, 5498, 9126, 2705, 2211, 4637, 1996, 2796, 2120, 3519, 2044, 3116, 3539, 2704, 1052, 1012, 1058, 1012, 27544, 5332, 2213, 3270, 10546, 1012, 2019, 5448, 8554, 4146, 2011, 1996, 2932, 13970, 12274, 17130, 10173, 2008, 3519, 1010, 2007, 11948, 5498, 9126, 2705, 1005, 1055, 2490, 1010, 2453, 2663, 2039, 2000, 7558, 4272, 1999, 1996, 6008, 10703, 3320, 1012, 1999, 2727, 1010, 2043, 1996, 3519, 2283, 2787, 2000, 25705, 2007, 2035, 2634, 4698, 2852, 18891, 2850, 14163, 10087, 6494, 10556, 27922, 16098, 2213, 1006, 9932, 4215, 2213, 2243, 1007, 2005, 1996, 3320, 2602, 1999, 6008, 10703, 1010, 11948, 5498, 9126, 2705, 2904, 8884, 7368, 1998, 3569, 1996, 2852, 18891, 2850, 14163, 10087, 6494, 10556, 27922, 16098, 2213, 1006, 1040, 2213, 2243, 1007, 1011, 6008, 5003, 7088, 2721, 3519, 1006, 1056, 12458, 1007, 4707, 1012, 1996, 1056, 12458, 2109, 1037, 10165, 2004, 2037, 2602, 6454, 1998, 2109, 2019, 3746, 1997, 11948, 5498, 9126, 2705, 5559, 1037, 10165, 2013, 1996, 2143, 4698, 9067, 4886, 1999, 2037, 14921, 1012, 11948, 5498, 9126, 2705, 2056, 1024, 1000, 2130, 2643, 3685, 3828, 6008, 10703, 2065, 9932, 4215, 2213, 2243, 5651, 2000, 2373, 1012, 1000, 11948, 5498, 9126, 2705, 2878, 27693, 2135, 3569, 1996, 1040, 2213, 2243, 1998, 1056, 12458, 4707, 1998, 2356, 1996, 2111, 1997, 6008, 10703, 1998, 2010, 4599, 2000, 3789, 2005, 2008, 4707, 1012, 2023, 4707, 2018, 1037, 3143, 3377, 1999, 2727, 1012, 11948, 5498, 9126, 2705, 2036, 3569, 1996, 1040, 2213, 2243, 1011, 1056, 12458, 4707, 1999, 1996, 5768, 2602, 1010, 2218, 1996, 2168, 2095, 1012, 2101, 1999, 2432, 1010, 11948, 5498, 9126, 2705, 2056, 2002, 2052, 7714, 3789, 2005, 1996, 24243, 20308, 2283, 1006, 24954, 1007, 2021, 2052, 2025, 7949, 2010, 2490, 2000, 2151, 2392, 2076, 1996, 9046, 2796, 2236, 2602, 1012, 1996, 2283, 1010, 2174, 1010, 3478, 2000, 2663, 2151, 4272, 1999, 6008, 10703, 1999, 1996, 13660, 11200, 1012, 4599, 1997, 11948, 5498, 9126, 2705, 1999, 6008, 10703, 2031, 10843, 15520, 2006, 102]



In [1]:
import tensorflow as tf

# tf.enable_eager_execution()
# layernorm = tf.keras.layers.LayerNormalization()


In [14]:
tmp= tf.convert_to_tensor(input_id, dtype=tf.float32)
int(tmp[3])

TypeError: int() argument must be a string, a bytes-like object or a number, not 'Tensor'

In [76]:
a = list(tmp.numpy())
type(a[0])

numpy.float32

In [65]:
lists = [2,2,3,4,5,6]
for i, l in enumerate(lists):
    print(i, l)

0 2
1 2
2 3
3 4
4 5
5 6


In [23]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for i in res:
        new = sess.run(i)
        print(new)


[  101  2339  2106  2002  2689  2010  2490  2000  2068  1029   102 17251
 13198  2031  2036  4427  2195  4684  5097  2005 16380  1998 11924  1012
  4401  1010  2107  2004 16480 14115 26760 24079  1010  2031  7034  2008
 11948  5498  9126  2705  2038  1996  4022  2000  2022  3144  1999  2796
  4331  2349  2000  2010  6217  1998  5470  2918  2894  1012  1999  2786
  1010 11948  5498  9126  2705  2211  4637  1996  2796  2120  3519  2044
  3116  3539  2704  1052  1012  1058  1012 27544  5332  2213  3270 10546
  1012  2019  5448  8554  4146  2011  1996  2932 13970 12274 17130 10173
  2008  3519  1010  2007 11948  5498  9126  2705  1005  1055  2490  1010
  2453  2663  2039  2000  7558  4272  1999  1996  6008 10703  3320  1012
  1999  2727  1010  2043  1996  3519  2283  2787  2000 25705  2007  2035
  2634  4698  2852 18891  2850 14163 10087  6494 10556 27922 16098  2213
  1006  9932  4215  2213  2243  1007  2005  1996  3320  2602  1999  6008
 10703  1010 11948  5498  9126  2705  2904  8884  7

In [11]:
input_tensor = tf.zeros([1,384,768])
history_output = tf.zeros([1,12,114,768])
history_mask = tf.zeros([12,114])
history_turns = 0

In [14]:
num1 = history_output.shape[1]
history_mask = tf.expand_dims(history_mask, axis=0)#1,12,114
history_list = tf.unstack(history_output, num1, axis=1)[:history_turns]#1,114,768
mask_list = tf.unstack(history_mask, num1, axis=1)[:history_turns]#1,114
result = []
for history, mask in zip(history_list, mask_list):
    # scores: batch x word_num1 x word_num2
    scores = scoring(input_tensor, history)#1 * 384 * 114
    num = tf.shape(scores)[1]
    # empty_mask = history_mask.eq(0).unsqueeze(1).expand_as(scores)
    history_mask = tf.expand_dims(mask, 1)#1，1，114
    empty_mask = tf.tile(history_mask, [1, num, 1])#矩阵扩张 12，12，384，114
    score = tf.cast(scores, tf.float32)
    empty_mask = tf.cast(empty_mask, tf.float32)

    score =  (score * empty_mask)#1,384,114

    # score = tf.unstack(score, axis=1)
    # n = len(score)
    # score = tf.concat([i for i in score], 1)
    # softmax
    # alpha = alpha_flat.view(-1, input_tensor.size(1), history_qa_ids.size(1))
    alpha_flat = tf.nn.softmax(score, axis=2)
    # alpha_flat = tf.split(alpha_flat, n, axis=1)
    # alpha = tf.stack([i for i in alpha_flat], axis=1)

    # alpha: batch x word_num1 x word_num2
    attended = tf.matmul(alpha_flat, x3)
    attended = tf.cast(attended, tf.float32)
    # attended = alpha.bmm(x3)
    # attended: batch x word_num1 x dim_3
    result.append(attended)
if result:
    mean = tf.zeros(tf.shape(input_tensor), dtype=tf.float32)
    for i in range(0, history_turns):
        mean += result[i]
    attend = mean / history_turns
else:
    attend = tf.zeros(tf.shape(input_tensor), dtype=tf.float32)
attend = tf.squeeze(attend, axis=0)


In [16]:
attend

<tf.Tensor 'Squeeze:0' shape=(384, 768) dtype=float32>